In [1]:
import json
import cv2
import numpy as np
from pycocotools.coco import COCO
from pathlib import Path
import os

# Đường dẫn tới thư mục chứa ảnh và file chú thích
data_dir = Path("D:/THUC_TAP/train2017")  # Cập nhật thành train2017
annotation_file = Path("D:/THUC_TAP/annotations/instances_train2017.json")  # Cập nhật thành train2017

# Thư mục để lưu ảnh sau khi xử lý
output_dir = Path("Project_test2/picture2")
os.makedirs(output_dir, exist_ok=True)

# Tải file chú thích
coco = COCO(annotation_file)

# Hàm tách toàn bộ vật thể và lưu ảnh vào thư mục riêng biệt cho mỗi ảnh gốc
def extract_and_save_all_objects(image_id):
    # Lấy thông tin của ảnh
    image_info = coco.loadImgs(image_id)[0]
    img_path = data_dir / image_info['file_name']
    img = cv2.imread(str(img_path))
    
    # Kiểm tra xem ảnh có được tải thành công không
    if img is None:
        return False  # Trả về False nếu không tải được
    
    # Tạo thư mục con cho ảnh gốc
    img_output_dir = output_dir / f"{image_info['file_name'].split('.')[0]}"
    os.makedirs(img_output_dir, exist_ok=True)

    # Lưu ảnh gốc vào thư mục con
    cv2.imwrite(str(img_output_dir / "original.png"), img)
    
    # Lấy mặt nạ cho tất cả các vật thể trong ảnh
    ann_ids = coco.getAnnIds(imgIds=image_id, iscrowd=0)
    anns = coco.loadAnns(ann_ids)
    
    for i, ann in enumerate(anns):
        # Tạo mask cho vật thể
        mask = coco.annToMask(ann) * 255
        object_img = cv2.bitwise_and(img, img, mask=mask.astype(np.uint8))
        
        # Lưu ảnh tách vật thể
        obj_output_path = img_output_dir / f"object_{i}.png"
        cv2.imwrite(str(obj_output_path), object_img)

        # Chuyển ảnh sang trắng đen
        bw_img = np.where(mask == 255, 255, 0).astype(np.uint8)
        bw_output_path = img_output_dir / f"object_{i}_bw.png"
        cv2.imwrite(str(bw_output_path), bw_img)
    
    return True  # Trả về True nếu xử lý thành công

# Thực hiện tách vật thể cho tất cả các ảnh
image_ids = coco.getImgIds()  # Lấy tất cả ID ảnh
successful_count = 0
for image_id in image_ids:
    if extract_and_save_all_objects(image_id):
        successful_count += 1

print(f"Đã xử lý thành công {successful_count} ảnh gốc.")


loading annotations into memory...
Done (t=12.64s)
creating index...
index created!
Đã xử lý thành công 11692 ảnh gốc.
